In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# SANS resolution estimate

In the small-angle limit, the basic quantity measured in a SANS instrument is the wavevector transfer
$$Q = \frac{4 \pi}{\lambda}\sin(\theta/2)\approx \frac{2\pi}{\lambda}\theta = k\theta$$
Here the angle between the incoming wavevector $k_i$ and the outgoing wavevector $k_f$ is $\theta$. Note that this differs from the $2\theta$ convention often used.

Some key instrument factors that need to be factored in when determining the SANS resolution are
- Wavelength distribution $f(\lambda)$
- Detector resolution, which is a function of
  - Pixel size
  - Wiring of detector
  - Possible scattering in layers (metal etc.) between vacuum chamber and detector gas/pixels
- Beam divergence distribution $f(\theta_{d})$
- Beam cross-section size

Another factor is the sample thickness $t$ (i.e. multiple scattering). This is hard to take into account and in practice for SANS as a technique to work, $t$ needs to be sufficiently small compared to $\Sigma$ so that multiple scattering can be neglected.

# 0. Preliminary definitions

In [ ]:
FZZ_map = {"Q1": 9742.34272, "Q2": 7427.9968, "Q3": 3422.98528, "Q4": 1432.00036}
sample_distances = (np.array(list(FZZ_map.values()))  + 1320) / 1e3
print(sample_distances)
half_height = 0.60 / 2 # m

def Q_from_theta(wavelength, theta):
    return 4*np.pi/wavelength*np.tan(theta/2)

## 1. $f(\lambda)$

Per the formula for $Q$, the distribution of wavelengths $\lambda$ can be expected to give a $\theta$-dependent resolution. As a rule, the error in $Q$ is linear in $\theta$ and as is the nominal value in the small-angle limit. 

In [ ]:
wavelength = 4 # AA
# dlambda of velocity selector
dlambda = 0.05 * wavelength # AA

In [ ]:
for d in sample_distances:
    max_angle = np.arctan(half_height/d) # rad
    max_angle_degrees = max_angle / (2 * np.pi) * 360 # deg
    print(max_angle, max_angle_degrees)
    angles = np.linspace(0, max_angle, 1000)
    min_lambda = wavelength - dlambda/2
    max_lambda = wavelength + dlambda/2

    max_Q = Q_from_theta(min_lambda,angles)
    min_Q = Q_from_theta(max_lambda,angles)
    Q_nominal = Q_from_theta(wavelength,angles)
    delta_Q = max_Q - min_Q
    plt.subplot(1,2,1)

    plt.plot(angles, delta_Q)
    plt.subplot(1,2,2)
    plt.plot(angles, Q_nominal)
    plt.show()

# 2. Detector characteristics

The per pixel resolution is determined by the solid angle $d\Omega \approx dA/L^2$ it takes up, which is a function of the distance from the sample $L$ and area $dA$.

In [ ]:
pixel_size = 0.001056338028169014 # m
pixel_area = pixel_size ** 2 # m2
for d in sample_distances:
    dOmega = pixel_area / d**2
    print(d,dOmega)

In an isotropic setting, the detector integrates radially and the true quantity of interest is $d\theta$, the angle element scattered into the pixel. This is found by $d\Omega \approx dh/L$ with $dh$ the pixel height. This means that as far as the pixels are concerned, in small angle approximation the angular resolution is constant for $\theta$.

In [ ]:
for d in sample_distances:
    dtheta = pixel_size / d
    print(d,dtheta)
    max_angle = np.arctan(half_height/d) # rad

In [ ]:
for d in sample_distances:
    dtheta = pixel_size / d
    dtheta_min = pixel_size / (np.sqrt(d**2 + half_height**2))
    print(d,dtheta,dtheta_min)
    max_angle = np.arctan(half_height/d) # rad
    dtheta_min = pixel_size / (d * np.cos(max_angle))
    print(dtheta_min)

## 3. Translating the wavelength distribution to a rebinning

There are two resolution expressions: the resolution due to wavelength decreases as the angle $\theta$ increases. As far as the pixels are concerned, the angular resolution is more or less constant as function $\theta$, the only deviations being proportional to the small-angle error. What remains is to estimate how for typical $\theta$ values, the error of about $5$\% in $Q$ can be translated to a rebinning to match this.

In [ ]:
for d in sample_distances:
    max_angle = np.arctan(half_height/d) # rad
    max_angle_degrees = max_angle / (2 * np.pi) * 360 # deg
    print(max_angle, max_angle_degrees)
    angles = np.linspace(0, max_angle, 1000)

    dtheta = pixel_size / d
    half_angle = max_angle / 2
    # Equivalent error from wavelength
    d_half_angle = 0.05 * angles
    print("Distance, dtheta and max_angle")
    print(d, dtheta, max_angle)
    plt.plot(angles,d_half_angle/dtheta)
    plt.ylabel(r'$u_\theta/d\theta_{pixel}$')
    plt.xlabel(r'$\theta$ [rad]')
    plt.grid()
    plt.show()

Translating the uncertainty in wavelength to one in Q shows that the effective resolution curves take the exact same values for every $d$ value: for lower angles, the wavelength uncertainty translates to a resolution of a few bins and for the highest angles, the uncertainty appears to give the equivalent of as much as 14 bins! From this it appears reasonable to rebin as $4\times 4$ or so, perhaps cutting off the most extreme angles/Q values due to the low resolution and high uncertainty there.

## X. Notes:
The wavelength and divergence distributions are not independent: it seems likely that the shape of the wavelength distribution is in fact a function of not only velocity selector characteristics but also the divergence of the incoming neutrons. This is not relevant for simple resolution estimates but could be a factor if exact expressions for resolution are required. 